To stop processes
```bash
pkill -9 -f pw.x
pkill -9 -f gipaw.x

```

In [ ]:
import f90nml
nml = f90nml.write('sample.nml')



In [ ]:
crystal=read("test/MIN-167-350K-CuCbPyz.cif")

In [ ]:
from ase.visualize import view
view(crystal)

In [ ]:
import os

print(os.getcwd())

In [ ]:
from ase.io import read, write
from ase.visualize import view
# crystal=read("../structures/KTU-183_2_auto.cif")
crystal=read("../structures/KTU-183_2_auto_dimer.xyz")
view(crystal)

In [ ]:
len(crystal)

# Main calculation

In [ ]:
%%time
%load_ext autoreload
%autoreload 2

from runner import pw_runner
from ase.io import read, write

import time

import os

# Defining parameters:

np_pw = 13
np_gipaw = 13

pw_params = {
    'prefix':'crystal', 
    'restart_mode' : 'from_scratch',
    'tstress':True, 
    'tprnfor':True, 
    'nosym':True, 
    'ecutwfc':20, 
    'kpts':(1, 1, 1),
    # 'kpts':None, 
    'ecutrho' : 200,
    # 'occupations' : 'smearing', 
    # 'smearing' : 'gauss', 
    # 'degauss' : 1.0e-2
}

# Loading structures:

# crystal=read("../structures/MIN-167-350K-CuCbPyz.cif")
# crystal=read("../structures/HIK-143 293K-activated.cif")
# crystal=read("../structures/HIK-143 MeOH.cif")
crystal=read("../structures/KTU-183_2_auto.cif")

tms = read("../structures/tms.xyz")
# crystal = tms
a = 10.0
tms.set_cell([a, a, a])
tms.set_cell([(a, 0, 0), (0, a, 0), (0, 0, a)])
tms.set_pbc(True)

# Creating directory to store all the data:
calc_dir = pw_runner.make_calc_dir()


print("Starting QE/SCF calculation for TMS...")
pw_runner.run_pw_scf(calc_dir, tms, num_proc_pw=np_pw, pw_params=pw_params)
print("Starting gipaw calculation for TMS...")
gipaw_out = pw_runner.run_gipaw(calc_dir, "espresso_gipaw_tms.pwo", num_proc_gipaw=np_gipaw)
chemical_shifts_iso, chemical_shifts_tensors = pw_runner.parse_gipaw_output(calc_dir + '/espresso_gipaw_tms.pwo', num_atoms=len(tms))
tms.info['chemical_shifts_iso'] = chemical_shifts_iso
tms.info['chemical_shifts_tensors'] = chemical_shifts_tensors
write(calc_dir+"/tms.xyz", tms, format='extxyz')
os.rename(calc_dir+"/espresso.pwo", calc_dir+"/espresso_tms.pwo")

print("Starting QE/SCF calculation for crystal...")
pw_runner.run_pw_scf(calc_dir, crystal, num_proc_pw=np_pw, pw_params=pw_params)
print("Starting gipaw calculation for crystal...")
gipaw_out = pw_runner.run_gipaw(calc_dir, "espresso_gipaw.pwo", num_proc_gipaw=np_gipaw)
chemical_shifts_iso, chemical_shifts_tensors = pw_runner.parse_gipaw_output(calc_dir + '/espresso_gipaw.pwo', num_atoms=len(crystal))
crystal.info['chemical_shifts_iso'] = chemical_shifts_iso
crystal.info['chemical_shifts_tensors'] = chemical_shifts_tensors
write(calc_dir+"/crystal.xyz", crystal, format='extxyz')

In [ ]:
from runner.plot import plot_chemical_shifts

path1 = "/media/dlb/la_cie1/repos/pw-benchmarks/qe/test/2023-01-10_17 40-57_007593"
path2 = "/media/dlb/la_cie1/repos/pw-benchmarks/qe/test/2023-01-11_18 05-03_816467"

crystal1 = read(path1+"/crystal.xyz", format="extxyz")
crystal2 = read(path2+"/crystal.xyz", format="extxyz")

ch1= plot_chemical_shifts(crystal1, "./test")
ch2 = plot_chemical_shifts(crystal2, "./test")

print(ch1['H']-ch2['H']) 
# print(ch1['C']-ch2['C'])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.arange(1, len(scf_times)+1, 1, dtype=int)
plt.plot(x, scf_times_5, label='SCF time')
plt.plot(x, gipaw_times_5, label='GIPAW time')
plt.xticks(x)
plt.xlabel('Num. of MPI processes')
plt.ylabel('Time, s.')
plt.legend()
plt.show()

In [ ]:
x

In [ ]:
print(start_time, scf_time, gipaw_time-start_time)

In [ ]:
struct = read(calc_dir+"/crystal.xyz", format="extxyz")
print( struct.get_potential_energy() )
print( struct.get_forces() )
print( struct.info['chemical_shifts_iso'])
print( struct.info['chemical_shifts_tensors'])

In [ ]:
import psutil
 
# Getting % usage of virtual_memory ( 3rd field)
print('RAM memory % used:', psutil.virtual_memory()[2])
# Getting usage of virtual_memory in GB ( 4th field)
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)

In [ ]:
os.chdir(head_dir)
crystal=read("test/MIN-167-350K-CuCbPyz.cif")

view(tms)


In [ ]:
print( crystal.get_cell() )
print( crystal.get_pbc() )

crystal=read("test/MIN-167-350K-CuCbPyz.cif")
crystal.get_forces()

In [ ]:
view(crystal)

# DFTB+

In [ ]:
# !ulimit -s unlimited
# conda env export > dftb_plumed.yml
# https://wiki.fysik.dtu.dk/ase/ase/calculators/dftb.html
# https://dftb.org/parameters/download

# export ASE_DFTB_COMMAND="/path/to/dftb+ > PREFIX.out"
# export DFTB_PREFIX=/path/to/mio-0-1/
import os

os.environ['OMP_NUM_THREADS'] = "5,1"
os.environ["ASE_DFTB_COMMAND"] = "ulimit -s unlimited; /home/dlb/anaconda3/envs/dftb_plumed/bin/dftb+"
os.environ["DFTB_PREFIX"] = "/home/dlb/Downloads/pbc-0-3"
# os.environ["DFTB_PREFIX"] = "/home/dlb/Downloads/auorgap-1-1" # doesn't work for now with crystal...


In [ ]:
%%time
# from ase.calculators.dftb import Dftb
from runner.dftb import Dftb
from ase.io import write, read
from ase.build import molecule
from ase.optimize import QuasiNewton

atoms = molecule('H2O')
crystal=read("../structures/KTU-183_2_auto.cif")
# crystal=read("../structures/HIK-143 293K-activated.cif")
# crystal=read("../structures/MIN-167-350K-CuCbPyz.cif")
atoms = crystal
atoms.set_pbc(True)

# calc = Dftb(atoms=atoms,
#             label='crystal',
#             Hamiltonian_ = "xTB",
#             Hamiltonian_Method = "GFN1-xTB",
#             # Hamiltonian_MaxAngularMomentum_='',
#             # Hamiltonian_MaxAngularMomentum_O='p',
#             # Hamiltonian_MaxAngularMomentum_H='s',
#             # Hamiltonian_MaxAngularMomentum_N='s',
#             # Hamiltonian_MaxAngularMomentum_C='s',
#             # Hamiltonian_MaxAngularMomentum_Si='s',
#             kpts=(1,1,1),
#         #     Hamiltonian_SCC='Yes',
#         #     SCCTolerance = 1e-5,
#         #     Options_='',
#             # Verbosity=0,
#             # Hamiltonian_OrbitalResolvedSCC = 'Yes',
#             # Hamiltonian_SCCTolerance=1e-15,
#             # kpts=None
#             # Driver_='ConjugateGradient',
#             # Driver_MaxForceComponent=1e-3,
#             # Driver_MaxSteps=200,
#             # Driver_LatticeOpt = 'Yes',
#             # Driver_AppendGeometries = 'Yes'
#             )


calc = Dftb(atoms=atoms,
            label='crystal',
            # Hamiltonian_ = "xTB",
            # Hamiltonian_Method = "GFN1-xTB",
            # Hamiltonian_MaxAngularMomentum_='',
            # Hamiltonian_MaxAngularMomentum_O='p',
            # Hamiltonian_MaxAngularMomentum_H='s',
            # Hamiltonian_MaxAngularMomentum_N='s',
            # Hamiltonian_MaxAngularMomentum_C='s',
            # Hamiltonian_MaxAngularMomentum_Si='s',
            kpts=(1,1,1),
            # Hamiltonian_SCC='Yes',
        #     SCCTolerance = 1e-5,
        #     Options_='',
            # Verbosity=0,
            # Hamiltonian_OrbitalResolvedSCC = 'Yes',
            # Hamiltonian_SCCTolerance=1e-15,
            # kpts=None
            # Driver_='ConjugateGradient',
            # Driver_MaxForceComponent=1e-3,
            # Driver_MaxSteps=200,
            # Driver_LatticeOpt = 'Yes',
            # Driver_AppendGeometries = 'Yes'
            )

atoms.calc = calc

print(atoms.get_forces())


# print(atoms.get_potential_energy())
# print(atoms.get_forces())

# calc.calculate(atoms)
# The 'geo_end.gen' file written by the ASE calculator
# (containing the initial geometry), has been overwritten
# by DFTB+ and now contains the final, optimized geometry.
# final = read('geo_end.gen')
# write('final.xyz', final)


# dyn = QuasiNewton(atoms, trajectory='test.traj')
# dyn.run(fmax=0.01)
# write('final.xyz', atoms)

In [ ]:
print(atoms.get_potential_energy())
print(atoms.get_forces())

In [ ]:
print(atoms.get_forces())


In [ ]:
from ase import Atom, Atoms
from ase.build import bulk
from ase.calculators.lammpsrun import LAMMPS

parameters = {'pair_style': 'eam/alloy',
              'pair_coeff': ['* * NiAlH_jea.eam.alloy H Ni']}

files = ['NiAlH_jea.eam.alloy']

Ni = bulk('Ni', cubic=True)
H = Atom('H', position=Ni.cell.diagonal()/2)
NiH = Ni + H

lammps = LAMMPS(parameters=parameters, files=files)

NiH.calc = lammps
print("Energy ", NiH.get_potential_energy())

In [ ]:
atoms.get_cell()

In [ ]:
traj = read('geo_end.xyz', index=":")
view(traj)

In [ ]:
%%time

# print(atoms.get_potential_energy())
print(atoms.get_forces())

In [ ]:
crystal.get_potential_energy()

In [ ]:
# crystal=read("../structures/HIK-143 MeOH.cif")
crystal=read("../structures/HIK-143 293K-activated.cif")

In [ ]:
atoms.get_pbc()

In [ ]:
from ase.visualize import view
view(crystal)

# Plumed

In [ ]:
# https://www.plumed.org/doc-v2.7/user-doc/html/_m_e_t_a_d.html
# https://github.com/dftbplus/dftbplus/issues/1064
# try with numpy 1.22　
# https://gitlab.com/Sucerquia/ase-plumed_tutorial

from ase.calculators.lj import LennardJones
# from ase.calculators.plumed import Plumed
from runner.plumed import Plumed
# from plumed import Plumed
from ase.constraints import FixedPlane
from ase.md.langevin import Langevin
from ase.io import read
from ase import units

# import plumed

from ase.calculators.dftb import Dftb


timestep = 0.1
ps = 1000 * units.fs

setup = [f"UNITS LENGTH=A TIME={1/ps} ENERGY={units.mol/units.kJ}",
        #  "d: DISTANCE ATOMS=1,2 ",
        "c1: CENTER ATOMS=47,40",
        "c2: CENTER ATOMS=41,47,40,46,42,48",
        "c3: CENTER ATOMS=52,62,86,171,98,74",
        "c4: CENTER ATOMS=74,98",
         "phi: TORSION ATOMS=c1,c2,c3,c4",
         "metad: METAD ARG=phi SIGMA=2.10 HEIGHT=30.50 BIASFACTOR=5 TEMP=300.0 PACE=1",
         # "ARG=d SIGMA=0.20,0.20 HEIGHT=1.20 BIASFACTOR=5 TEMP=300.0 PACE=500",
         # "GRID_MIN=0,0 GRID_MAX=1.0,1.0 GRID_BIN=150,150",
         "PRINT ARG=phi STRIDE=1 FILE=test/plumed/COLVAR",
         # "CALC_RCT ",
         # "RCT_USTRIDE=10",
         # "...",
         "FLUSH STRIDE=1000"]

# setup = f"""
# d: DISTANCE ATOMS=1,2 
# metad: METAD ...
#    ARG=d SIGMA=0.20,0.20 HEIGHT=1.20 BIASFACTOR=5 TEMP=300.0 PACE=500 
#    GRID_MIN=0,0 GRID_MAX=1.0,1.0 GRID_BIN=150,150 
#    CALC_RCT 
#    RCT_USTRIDE=10 
# ...
# PRINT ARG=d STRIDE=100 FILE=COLVAR
# """
# print(setup)

from ase.build import molecule


# atoms = molecule('CH3CH2NH2')
# atoms = read('../structures/Ar_isomer.xyz')
# atoms = read('../structures/KTU-183_2_auto.cif')
atoms=read("../structures/KTU-183_2_auto_dimer.xyz")
# cons = [FixedPlane(i, [0, 0, 1]) for i in range(7)]
# atoms.set_constraint(cons)
# atoms.set_masses([1, 1, 1, 1, 1, 1, 1])

# calc_base = LennardJones(rc=2.5, r0=3.)

import os

os.environ["ASE_DFTB_COMMAND"] = "/home/dlb/anaconda3/envs/qe/bin/dftb+"
os.environ["DFTB_PREFIX"] = "/home/dlb/Downloads/pbc-0-3"

calc_base = Dftb(atoms=atoms,
            label='crystal',
            # Hamiltonian_ = "xTB",
            # Hamiltonian_Method = "GFN1-xTB",
            # Hamiltonian_MaxAngularMomentum_='',
            # Hamiltonian_MaxAngularMomentum_O='p',
            # Hamiltonian_MaxAngularMomentum_H='s',
            # Hamiltonian_MaxAngularMomentum_N='s',
            # Hamiltonian_MaxAngularMomentum_C='s',
            # Hamiltonian_MaxAngularMomentum_Si='s',
            kpts=(1,1,1),
            # Hamiltonian_SCC='Yes',
            # Verbosity=0,
            # Hamiltonian_OrbitalResolvedSCC = 'Yes',
            # Hamiltonian_SCCTolerance=1e-15,
            # kpts=None
            # Driver_='ConjugateGradient',
            # Driver_MaxForceComponent=1e-3,
            # Driver_MaxSteps=200,
            # Driver_LatticeOpt = 'Yes',
            # Driver_AppendGeometries = 'Yes'
            )

atoms.calc = Plumed(calc=calc_base,
                    input=setup,
                    timestep=timestep,
                    atoms=atoms,
                    kT=0.1,
                    log='test/plumed/plumed_log.log')

# from ase.md.verlet import VelocityVerlet

# atoms.calc = calc_base

# calc=LennardJones(rc=2.5, r0=3.)
# atoms.calc = calc

dyn = Langevin(atoms, timestep, temperature_K=0.1/units.kB, friction=1,
               fixcm=False, trajectory='test/plumed/UnbiasMD.xyz')
# We want to run MD with constant energy using the VelocityVerlet algorithm.
# dyn = VelocityVerlet(atoms, 1 * units.fs, trajectory='test/plumed/UnbiasMD.xyz')  # 5 fs time step.

dyn.run(1000)


In [ ]:
from ase.io import read
from ase.visualize import view

traj = read('test/plumed/UnbiasMD.xyz', index=":")
view(traj)

In [ ]:
from ase.io import write
write('test/plumed/traj.xyz', traj, format='xyz')

In [ ]:
traj = read('test/plumed/traj.xyz', index="0:3000")
view(traj)

In [ ]:
%%bash 
echo $A

In [ ]:
%%bash

export A=33
echo $A

In [ ]:
import numpy as np

from ase.build import molecule

import torch

atoms = molecule('CH3CH2NH2')

store_data = {
        'aaa': np.random.randn(300, 300),
        'bbb': "hello"
}


torch_data = torch.rand(2000, 3000)

atoms.info['store_data'] = store_data

atoms.info['torch_data'] = torch_data



%store atoms

In [ ]:
# $HOME/.ipython/profile_default/db/autorestore
%store


In [ ]:
%store 